In [27]:
#Resnet-50 PCA
import cv2
import os
import glob
import numpy as np 
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from pathlib import Path

In [28]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers     
from tensorflow.keras.layers import Activation, Dense, Flatten,Input,Conv2D,InputLayer
from tensorflow.keras.optimizers import RMSprop,Adam,SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator , img_to_array
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.regularizers import l2

In [29]:
from sklearn.decomposition import PCA,IncrementalPCA
from sklearn.metrics import accuracy_score
from sklearn import svm, metrics, datasets
from sklearn.model_selection import GridSearchCV, train_test_split
from skimage.io import imread
from skimage.transform import resize
from skimage.color import rgb2gray
from imutils import paths

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [31]:
def pcaclaheimg(image):
    image = np.array(image, dtype=np.uint8)
    image = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.3, tileGridSize=(8,8))
    image=clahe.apply(image)
    return 

In [68]:
def pcaimg(image):
    blue, green, red = cv2.split(image)
    df_blue = blue/255
    df_green = green/255
    df_red = red/255
    
    pca_b = PCA(n_components=50)
    pca_b.fit(df_blue)
    trans_pca_b = pca_b.transform(df_blue)
    
    pca_g = PCA(n_components=50)
    pca_g.fit(df_green)
    trans_pca_g = pca_g.transform(df_green)
    pca_r = PCA(n_components=50)
    pca_r.fit(df_red)
    trans_pca_r = pca_r.transform(df_red)    
    b_arr = pca_b.inverse_transform(trans_pca_b)
    g_arr = pca_g.inverse_transform(trans_pca_g)
    r_arr = pca_r.inverse_transform(trans_pca_r)
    img_reduced = (cv2.merge((b_arr, g_arr, r_arr)))
    return img_reduced

In [71]:
#Data Aug using Image Data Generator 
dir_path='./path/Dataset/group/'
size= 128
train= ImageDataGenerator( #shear_range = 0.2,
                           #rescale=1./255, 
                           validation_split=0.3,
                           rotation_range=5,
                           #brightness_range=[0.2,0.5],
                           #zoom_range=[0.0,0.1],
                           preprocessing_function=pcaimg
                          )

validation = ImageDataGenerator(#shear_range = 0.2,
                                #rescale=1./255,
                                validation_split=0.3,
                                rotation_range=5,
                                #brightness_range=[0.2,0.5],
                                #zoom_range=[0.0,0.1],
                                preprocessing_function=pcaimg
                                )

In [72]:
train_dataset= train.flow_from_directory(dir_path,
                                         target_size=(size,size),
                                         batch_size=23,
                                         color_mode="rgb",
                                         subset="training",
                                         class_mode="categorical")

validation_dataset= validation.flow_from_directory(dir_path,
                                         target_size=(size,size),  #240,180
                                         color_mode="rgb",
                                         batch_size=18,
                                         subset="validation",
                                         class_mode="categorical")

Found 220 images belonging to 20 classes.
Found 80 images belonging to 20 classes.


In [73]:
print("Train \n",train_dataset.classes)
print('validation \n',validation_dataset.classes)
n_classes= len(train_dataset.class_indices)
print(n_classes)

Train 
 [ 0  0  0  0  0  0  0  0  0  0  0  1  1  1  1  1  1  1  1  1  1  1  2  2
  2  2  2  2  2  2  2  2  2  3  3  3  3  3  3  3  3  3  3  3  4  4  4  4
  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5  5  5  5  6  6  6  6  6  6
  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  7  8  8  8  8  8  8  8  8
  8  8  8  9  9  9  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10 10 10 10
 10 11 11 11 11 11 11 11 11 11 11 11 12 12 12 12 12 12 12 12 12 12 12 13
 13 13 13 13 13 13 13 13 13 13 14 14 14 14 14 14 14 14 14 14 14 15 15 15
 15 15 15 15 15 15 15 15 16 16 16 16 16 16 16 16 16 16 16 17 17 17 17 17
 17 17 17 17 17 17 18 18 18 18 18 18 18 18 18 18 18 19 19 19 19 19 19 19
 19 19 19 19]
validation 
 [ 0  0  0  0  1  1  1  1  2  2  2  2  3  3  3  3  4  4  4  4  5  5  5  5
  6  6  6  6  7  7  7  7  8  8  8  8  9  9  9  9 10 10 10 10 11 11 11 11
 12 12 12 12 13 13 13 13 14 14 14 14 15 15 15 15 16 16 16 16 17 17 17 17
 18 18 18 18 19 19 19 19]
20


In [79]:
resnet_model= Sequential()
pretrained_model=tf.keras.applications.ResNet50(
      include_top=False, 
      weights='imagenet',
      input_tensor=None,
      input_shape=(128,128,3),
      pooling='max',
      classes=108
)

# Every layer inside that pretrained module is not Trainable unless True 
# Enable to Unlearn or Improve Accuracy  
for layer in pretrained_model.layers[90:]:
      layer.trainable=True
resnet_model.add(pretrained_model)
resnet_model.add(tf.keras.layers.Flatten())
resnet_model.add(tf.keras.layers.Dense(1008, activation='relu'))
#resnet_model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
resnet_model.add(Dense(20, kernel_regularizer=tf.keras.regularizers.l2(0.01),activation='softmax'))

In [80]:
resnet_model.load_weights('./svm1_weights.h5')

In [ ]:
layer_names=[layer.name for layer in pretrained_model.layers]
print(layer_names)

In [ ]:
'''model = Sequential()
#
model.add(Conv2D(input_shape=(240,180,1),
                 filters=64,  kernel_size=(3,3), padding="same", activation="relu"))

model.add(Conv2D(filters=64,  kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
#
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
#
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="sigmoid"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="sigmoid"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="sigmoid"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
#
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
#
model.add(Flatten())
model.add(Dense(units=1008,activation="relu"))
model.add(Dense(units=1008,activation="relu"))
#model.add(Dense(units=1008,activation="sigmoid"))
model.add(Dense(units=n_classes, activation="softmax"))'''

In [ ]:
resnet_model.summary()


In [ ]:
'''#LOAD WEIGHTS
resnet_model.load_weights('/content/drive/MyDrive/model/40/40model_weights.h5')
#POP LAYERS 
resnet_model.pop()
#new set of labels
resnet_model.add(Dense(n_classes,activation='softmax')) 
resnet_model.summary()'''

In [83]:
#optimizer Tuning
tf.keras.optimizers.Adam(
                            learning_rate=0.001,
                            beta_1=0.98,
                            beta_2=0.97,
                            epsilon=1e-07,
                            amsgrad=False,
                            name="Adam",
                      )

resnet_model.compile(
                            optimizer='Adam',
                            loss='squared_hinge',
                            metrics=['accuracy']
                    )

R50_model= resnet_model.fit(train_dataset,
                            #steps_per_epoch=30,
                            epochs=300, 
                            validation_data= validation_dataset,      
                            #callbacks=[checkpoint,],
                            verbose=1,
                            )  
                 
R50_model.model.save('svm1.h5')
R50_model.model.save_weights('svm1_weights.h5')  

Epoch 1/300
10/10 [==============================] - 102s 10s/step - loss: 0.9728 - accuracy: 0.9500 - val_loss: 0.9737 - val_accuracy: 0.8875
Epoch 2/300
10/10 [==============================] - 94s 9s/step - loss: 0.9803 - accuracy: 0.8364 - val_loss: 1.0048 - val_accuracy: 0.7375
Epoch 3/300
10/10 [==============================] - 82s 8s/step - loss: 0.9781 - accuracy: 0.8727 - val_loss: 1.0692 - val_accuracy: 0.4000
Epoch 4/300
10/10 [==============================] - 83s 8s/step - loss: 0.9761 - accuracy: 0.8864 - val_loss: 1.0719 - val_accuracy: 0.3875
Epoch 5/300
10/10 [==============================] - 81s 8s/step - loss: 0.9905 - accuracy: 0.8136 - val_loss: 1.0628 - val_accuracy: 0.4125
Epoch 6/300
10/10 [==============================] - 75s 7s/step - loss: 1.0100 - accuracy: 0.7045 - val_loss: 1.1161 - val_accuracy: 0.1750
Epoch 7/300
10/10 [==============================] - 81s 8s/step - loss: 1.0171 - accuracy: 0.6818 - val_loss: 1.1452 - val_accuracy: 0.0500
Epoch 8/300

In [78]:
resnet_model.save('./path/CNN_SVM_model.h5')
resnet_model.save_weights('./path/CNN_SVM_weight.h5')

In [ ]:
# plot the loss
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')